In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import psycopg2

In [2]:
url = "https://web.archive.org/web/20200318083015/https://en.wikipedia.org/wiki/List_of_largest_banks"
scrapped_data = requests.get(url)

In [3]:
print(scrapped_data.ok)

True


In [4]:
soup = BeautifulSoup(scrapped_data.text, 'html.parser')

In [5]:
data_list = []

headline = soup.find(id="By_market_capitalization")
table = headline.find_next("table")
doc = table.tbody

for element in doc.contents[2::2]:
    anchors = element.find_all('a')
    name = anchors[-1].text
    price = element.find_all('td')[-1].text
    price = float(price[:-1])
    data_list.append({"Name": name, "Market Cap (US$ Billion)": price})

data = pd.DataFrame(data_list)
data

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
65,Ping An Bank,37.993
66,Standard Chartered,37.319
67,United Overseas Bank,35.128
68,QNB Group,33.560


In [15]:
def create_db(dbname):

    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    cur.execute("DROP DATABASE IF EXISTS " + dbname)
    cur.execute("CREATE DATABASE " + dbname)

    conn.close()

    conn = psycopg2.connect("host=127.0.0.1 dbname=" + dbname + " user=postgres password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    return conn, cur

In [17]:
dbname = "banks"
conn, cur = create_db(dbname)

In [18]:
largest_banks_create = ("""CREATE TABLE IF NOT EXISTS largest_banks(
country_name varchar,
market_cap numeric
)"""
)

In [19]:
cur.execute(largest_banks_create)

In [25]:
largest_banks_insert = ("""INSERT INTO largest_banks(
country_name,
market_cap)
VALUES(%s, %s)
""")

In [29]:
for i, row in data.iterrows():
    cur.execute(largest_banks_insert, list(row))


In [30]:
conn.close()
cur.close()